# **Added tversky loss, combined with dice loss+ augmentations, tc focused croping, mix up**

In [1]:
pip install tensorflow

In [2]:
!pip install nibabel

In [3]:
!pip show nibabel

Name: nibabel
Version: 5.3.2
Summary: Access a multitude of neuroimaging data formats
Home-page: https://nipy.org/nibabel
Author: 
Author-email: NiBabel developers <neuroimaging@python.org>
License: MIT License
Location: /usr/local/lib/python3.12/dist-packages
Requires: numpy, packaging, typing-extensions
Required-by: 


**Importing libraries**

In [3]:
#cell 1

import tensorflow as tf
import os
import numpy as np
import matplotlib.pyplot as plt
import nibabel as nib
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Conv3DTranspose, Concatenate, Input,LeakyReLU,UpSampling3D
from tensorflow.keras.models import Model





Defining instance normalization instead of batch normalization(due to low batch size)

In [4]:
#cell 2

import tensorflow as tf
from tensorflow.keras.layers import Layer

class InstanceNormalization(Layer):
    def __init__(self, epsilon=1e-5, **kwargs):
        super(InstanceNormalization, self).__init__(**kwargs)
        self.epsilon = epsilon

    def build(self, input_shape):
        # Trainable parameters: gamma (scale) and beta (shift)
        self.gamma = self.add_weight(
            name='gamma',
            shape=(input_shape[-1],),
            initializer='ones',
            trainable=True
        )
        self.beta = self.add_weight(
            name='beta',
            shape=(input_shape[-1],),
            initializer='zeros',
            trainable=True
        )
        super(InstanceNormalization, self).build(input_shape)

    def call(self, inputs):
        mean, variance = tf.nn.moments(inputs, axes=[1, 2, 3], keepdims=True)
        normalized = (inputs - mean) / tf.sqrt(variance + self.epsilon)
        return self.gamma * normalized + self.beta


In [5]:
#cell 3
from google.colab import drive
# Check if drive is already mounted
try:
    drive.mount('/content/drive', force_remount=True) # Adding force
except ValueError:
    # If already mounted, skip mounting
    print("Drive is already mounted.")
    pass  # Or you could raise a warning if preferred


Mounted at /content/drive


loading nifti files using nibabel
using z score normalization with optional mask
resize the image,segmentation mask(convert seg mask into one hot encoding)


In [6]:
#cell 4
import tensorflow as tf
import numpy as np
from scipy.ndimage import map_coordinates

import tensorflow as tf
import numpy as np
import nibabel as nib

def Load_nifti(data_path):
    """Load a NIfTI file and return its data as numpy array."""
    img = nib.load(data_path)
    img_data = img.get_fdata()
    return img_data

def z_score_normalize(image, mask=None):
    """Z-score normalization with optional masking to compute statistics only from specific regions"""
    if mask is not None:
        valid_pixels = image[mask > 0]
        if valid_pixels.size == 0:
            valid_pixels = image.flatten()  # Fallback to all pixels if mask is empty
    else:
        valid_pixels = image.flatten()
    mean = np.mean(valid_pixels)
    std = np.std(valid_pixels)
    if std == 0:
        std = 1.0  # Prevent division by zero
    return (image - mean) / std

def resize_image(image, target_shape=(128, 128, 128), method='bilinear'):
    """Resize a 3D image to target shape using TensorFlow operations."""
    if not isinstance(image, tf.Tensor):
        image = tf.convert_to_tensor(image, dtype=tf.float32)
    if len(image.shape) == 3:
        image = tf.expand_dims(image, axis=-1)#compatable with tf.image.resize
    image = tf.image.resize(image, [target_shape[0], target_shape[1]], method=method)
    image = tf.transpose(image, [2, 0, 1, 3])
    image = tf.image.resize(image, [target_shape[2], target_shape[1]], method=method)
    image = tf.transpose(image, [1, 2, 0, 3])
    image = image[:target_shape[0], :target_shape[1], :target_shape[2], :]# ensuring the
    return image



def preprocess_segmentation_mask(seg, target_shape=(128, 128, 128), deep_supervision=False):
    """Resize and one-hot encode segmentation mask."""

    if not isinstance(seg, tf.Tensor):
        seg = tf.convert_to_tensor(seg, dtype=tf.float32)
    if len(seg.shape) == 3:
        seg = tf.expand_dims(seg, axis=-1)

    seg = tf.image.resize(seg, [target_shape[0], target_shape[1]], method='nearest')
    seg = tf.transpose(seg, [2, 0, 1, 3])
    seg = tf.image.resize(seg, [target_shape[2], target_shape[1]], method='nearest')
    seg = tf.transpose(seg, [1, 2, 0, 3])
    seg = seg[:target_shape[0], :target_shape[1], :target_shape[2], :]
    seg_int = tf.cast(tf.round(seg), tf.int32)
    seg_int = tf.squeeze(seg_int, axis=-1)
    seg_one_hot = tf.one_hot(seg_int, depth=4)
    seg_one_hot = tf.ensure_shape(seg_one_hot, (target_shape[0], target_shape[1], target_shape[2], 4))

    if deep_supervision:
        seg_out2 = resize_image(seg, (64, 64, 64), method='nearest')
        seg_out2 = tf.cast(tf.round(seg_out2), tf.int32)
        seg_out2 = tf.squeeze(seg_out2, axis=-1)
        seg_out2 = tf.one_hot(seg_out2, depth=4)
        seg_out2 = tf.ensure_shape(seg_out2, (64, 64, 64, 4))
        seg_out3 = resize_image(seg, (32, 32, 32), method='nearest')
        seg_out3 = tf.cast(tf.round(seg_out3), tf.int32)
        seg_out3 = tf.squeeze(seg_out3, axis=-1)
        seg_out3 = tf.one_hot(seg_out3, depth=4)
        seg_out3 = tf.ensure_shape(seg_out3, (32, 32, 32, 4))
        seg_out4 = resize_image(seg, (16, 16, 16), method='nearest')
        seg_out4 = tf.cast(tf.round(seg_out4), tf.int32)
        seg_out4 = tf.squeeze(seg_out4, axis=-1)
        seg_out4 = tf.one_hot(seg_out4, depth=4)
        seg_out4 = tf.ensure_shape(seg_out4, (16, 16, 16, 4))
        return (seg_one_hot, seg_out2, seg_out3, seg_out4)
    return seg_one_hot

def preprocess_sample(t1c, t1n, t2f, t2w, seg=None, target_shape=(128, 128, 128)):
    """Normalize, resize, and prepare MRI images and segmentation masks with Z-score normalization."""
    threshold = np.percentile(t1c, 5)
    brain_mask = (t1c > threshold)
    t1c_norm = z_score_normalize(t1c, brain_mask)
    t1n_norm = z_score_normalize(t1n, brain_mask)
    t2f_norm = z_score_normalize(t2f, brain_mask)
    t2w_norm = z_score_normalize(t2w, brain_mask)
    t1c_resized = resize_image(t1c_norm, target_shape)
    t1n_resized = resize_image(t1n_norm, target_shape)
    t2f_resized = resize_image(t2f_norm, target_shape)
    t2w_resized = resize_image(t2w_norm, target_shape)
    t1c_final = tf.squeeze(t1c_resized, axis=-1)
    t1n_final = tf.squeeze(t1n_resized, axis=-1)
    t2f_final = tf.squeeze(t2f_resized, axis=-1)
    t2w_final = tf.squeeze(t2w_resized, axis=-1)
    image_tensor = tf.stack([t1c_final, t1n_final, t2f_final, t2w_final], axis=-1)
    return image_tensor
import tensorflow as tf



@tf.function
def augment_mri_sample(image, seg_tensor):
    """Apply stable data augmentation to MRI images and segmentation masks to improve Tumor Core (TC) segmentation."""
    # Handle deep supervision: unpack tuple of segmentation masks if provided
    if isinstance(seg_tensor, tuple):
        seg_one_hot, seg_out2, seg_out3, seg_out4 = seg_tensor
    else:
        seg_one_hot = seg_tensor

    # Helper function to resize tensors back to original shape after cropping
    def resize_to_original(tensor, target_shape=(128, 128, 128), is_seg=False):
        if len(tensor.shape) == 4:  # Image or segmentation (H, W, D, C)
            # Resize XY plane
            tensor = tf.image.resize(tensor, target_shape[:2], method='bilinear' if not is_seg else 'nearest')
            # Transpose to resize Z dimension
            tensor = tf.transpose(tensor, [2, 0, 1, 3])
            tensor = tf.image.resize(tensor, [target_shape[2], target_shape[1]], method='bilinear' if not is_seg else 'nearest')
            # Restore original axis order
            tensor = tf.transpose(tensor, [1, 2, 0, 3])
        return tensor

    # TC-focused Random Cropping to emphasize TC boundaries
    if tf.random.uniform(()) > 0.4:  # 60% probability to prioritize TC
        tc_mask = seg_one_hot[..., 1]  # Extract TC channel
        if tf.reduce_sum(tc_mask) > 0:  # Ensure TC is present
            # Compute TC region center for cropping
            coords = tf.where(tc_mask > 0)
            center = tf.reduce_mean(tf.cast(coords, tf.float32), axis=0)
            crop_size = 96  # Crop to 96x96x96 to focus on TC
            start = tf.cast(center - crop_size // 2, tf.int32)
            start = tf.clip_by_value(start, 0, 128 - crop_size)
            # Crop image and segmentation
            image = image[start[0]:start[0]+crop_size, start[1]:start[1]+crop_size, start[2]:start[2]+crop_size, :]
            seg_one_hot = seg_one_hot[start[0]:start[0]+crop_size, start[1]:start[1]+crop_size, start[2]:start[2]+crop_size, :]
            # Resize back to 128x128x128
            image = resize_to_original(image, is_seg=False)
            seg_one_hot = resize_to_original(seg_one_hot, is_seg=True)
            # Update auxiliary outputs for deep supervision
            if isinstance(seg_tensor, tuple):
                seg_out2 = resize_to_original(seg_one_hot, target_shape=(64, 64, 64), is_seg=True)
                seg_out3 = resize_to_original(seg_one_hot, target_shape=(32, 32, 32), is_seg=True)
                seg_out4 = resize_to_original(seg_one_hot, target_shape=(16, 16, 16), is_seg=True)

    # Random Flipping to balance with other augmentations
    if tf.random.uniform(()) > 0.6:  # 40% probability
        axis = tf.random.uniform((), minval=0, maxval=3, dtype=tf.int32)
        image = tf.reverse(image, axis=[axis])
        seg_one_hot = tf.reverse(seg_one_hot, axis=[axis])
        if isinstance(seg_tensor, tuple):
            seg_out2 = tf.reverse(seg_out2, axis=[axis])
            seg_out3 = tf.reverse(seg_out3, axis=[axis])
            seg_out4 = tf.reverse(seg_out4, axis=[axis])

    # Intensity Scaling to preserve TC contrast
    if tf.random.uniform(()) > 0.5:  # 50% probability
        intensity_factor = tf.random.uniform((), minval=0.95, maxval=1.05)
        image = image * intensity_factor

    # Gaussian Noise for robustness
    if tf.random.uniform(()) > 0.4:  # 60% probability
        noise_level = tf.random.uniform((), minval=0.0, maxval=0.02)
        noise = tf.random.normal(shape=tf.shape(image), mean=0.0, stddev=noise_level)
        image = image + noise
        image = tf.clip_by_value(image, -3.0, 3.0)

    # Ensure correct data types
    image = tf.convert_to_tensor(image, dtype=tf.float32)
    seg_one_hot = tf.convert_to_tensor(seg_one_hot, dtype=tf.float32)
    if isinstance(seg_tensor, tuple):
        seg_out2 = tf.convert_to_tensor(seg_out2, dtype=tf.float32)
        seg_out3 = tf.convert_to_tensor(seg_out3, dtype=tf.float32)
        seg_out4 = tf.convert_to_tensor(seg_out4, dtype=tf.float32)
        return image, (seg_one_hot, seg_out2, seg_out3, seg_out4)
    return image, seg_one_hot

In [7]:
#cell 5
import os
import numpy as np
import nibabel as nib
import tensorflow as tf

def data_generator(patient_folders, data_path, is_training=True, target_shape=(128, 128, 128)):
    for i, patient_id in enumerate(patient_folders):
        if i % 50 == 0:
            print(f"Step {i+1}/{len(patient_folders)}: Processing {patient_id}")
        try:
            t1c_path = os.path.join(data_path, patient_id, f"{patient_id}-t1c.nii.gz")
            t1n_path = os.path.join(data_path, patient_id, f"{patient_id}-t1n.nii.gz")
            t2f_path = os.path.join(data_path, patient_id, f"{patient_id}-t2f.nii.gz")
            t2w_path = os.path.join(data_path, patient_id, f"{patient_id}-t2w.nii.gz")
            if not all(os.path.exists(p) for p in [t1c_path, t1n_path, t2f_path, t2w_path]):
                print(f"Skipping {patient_id}: Missing MRI files")
                continue
            t1c = Load_nifti(t1c_path)
            t1n = Load_nifti(t1n_path)
            t2f = Load_nifti(t2f_path)
            t2w = Load_nifti(t2w_path)
            image_tensor = preprocess_sample(t1c, t1n, t2f, t2w, target_shape=target_shape)
            seg_path = os.path.join(data_path, patient_id, f"{patient_id}-seg.nii.gz")
            if not os.path.exists(seg_path):
                if is_training:
                    print(f"Skipping {patient_id}: Missing segmentation file")
                    continue
                else:
                    print(f"Generating all-zero mask for {patient_id} in validation mode")
                    seg = np.zeros((target_shape[0], target_shape[1], target_shape[2]), dtype=np.float32)
            else:
                seg = Load_nifti(seg_path)
            seg_tensor = preprocess_segmentation_mask(seg, target_shape, deep_supervision=is_training)
            if not is_training:
                # For validation, return a tuple of four tensors to match deep supervision structure
                seg_one_hot = seg_tensor
                seg_out2 = resize_image(seg_one_hot, (64, 64, 64), method='nearest')
                seg_out2 = tf.ensure_shape(seg_out2, (64, 64, 64, 4))

                seg_out3 = resize_image(seg_one_hot, (32, 32, 32), method='nearest')
                seg_out3 = tf.ensure_shape(seg_out3, (32, 32, 32, 4))

                seg_out4 = resize_image(seg_one_hot, (16, 16, 16), method='nearest')
                seg_out4 = tf.ensure_shape(seg_out4, (16, 16, 16, 4))

                seg_tensor = (seg_one_hot, seg_out2, seg_out3, seg_out4)

            if is_training:
                image_tensor, seg_tensor = augment_mri_sample(image_tensor, seg_tensor)
            # Debug shapes and sample values
            #print(f"{patient_id} image shape: {image_tensor.shape}, seg shape: {[s.shape for s in seg_tensor] if isinstance(seg_tensor, tuple) else seg_tensor.shape}")
            yield image_tensor, seg_tensor
        except Exception as e:
            print(f"Error processing {patient_id}: {e}")
            continue

In [ ]:
#cell 6

# Get all patient IDs
all_patients = os.listdir(train_data_path)
all_patients.sort()

# Remove any non-directory items
all_patients = [p for p in all_patients if os.path.isdir(os.path.join(train_data_path, p))]

# Stratified split if segmentation labels are available for all patients
train_val_patients, test_patients = train_test_split(all_patients, test_size=0.15, random_state=42)
train_patients, val_patients = train_test_split(train_val_patients, test_size=0.15 / 0.85, random_state=42)

print(f"Training: {len(train_patients)} patients")
print(f"Validation: {len(val_patients)} patients")
print(f"Test: {len(test_patients)} patients")

Training: 945 patients
Validation: 203 patients
Test: 203 patients


In [8]:
cell 7
import os
import glob
from sklearn.model_selection import train_test_split

train_data_path = "/content/drive/MyDrive/brats_goat_2024/MICCAI2024-BraTS-GoAT-TrainingData-With-GroundTruth"

# ✅ Safer way than os.listdir (avoids connection aborts on Drive)
all_patients = sorted([os.path.basename(p) for p in glob.glob(train_data_path + "/*") if os.path.isdir(p)])

print(f"Found {len(all_patients)} patients in dataset folder.")

# Split into train/val/test
train_val_patients, test_patients = train_test_split(all_patients, test_size=0.15, random_state=42)
train_patients, val_patients = train_test_split(train_val_patients, test_size=0.15/0.85, random_state=42)

print(f"Training: {len(train_patients)} patients")
print(f"Validation: {len(val_patients)} patients")
print(f"Test: {len(test_patients)} patients")

Found 1351 patients in dataset folder.
Training: 945 patients
Validation: 203 patients
Test: 203 patients


In [9]:

#cell 8
import tensorflow as tf
from tensorflow.keras import backend as K

# --- Metrics for Original Classes (Channels: 0:BG, 1:NCR, 2:ED, 3:ET) ---

class DiceET(tf.keras.metrics.Metric):
    def __init__(self, name='dice_et', **kwargs):
        super(DiceET, self).__init__(name=name, **kwargs)
        self.intersection = self.add_weight(name='dice_et_intersection', initializer='zeros')
        self.total = self.add_weight(name='dice_et_total', initializer='zeros')
        self.built = True

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true_region = tf.cast(y_true[..., 3], tf.float32)
        y_pred_region = tf.cast(y_pred[..., 3] > 0.5, tf.float32)
        self.intersection.assign_add(tf.reduce_sum(y_true_region * y_pred_region))
        self.total.assign_add(tf.reduce_sum(y_true_region) + tf.reduce_sum(y_pred_region))

    def result(self):
        dice = (2. * self.intersection + K.epsilon()) / (self.total + K.epsilon())
        return tf.where(tf.equal(self.total, 0), tf.where(tf.equal(self.intersection, 0), 1.0, 0.0), dice)

    def reset_state(self):
        self.intersection.assign(0.0)
        self.total.assign(0.0)

    def reset_states(self):
        self.reset_state()

class DiceNCR(tf.keras.metrics.Metric):
    def __init__(self, name='dice_ncr', **kwargs):
        super(DiceNCR, self).__init__(name=name, **kwargs)
        self.intersection = self.add_weight(name='dice_ncr_intersection', initializer='zeros')
        self.total = self.add_weight(name='dice_ncr_total', initializer='zeros')
        self.built = True

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true_region = tf.cast(y_true[..., 1], tf.float32)
        y_pred_region = tf.cast(y_pred[..., 1] > 0.5, tf.float32)
        self.intersection.assign_add(tf.reduce_sum(y_true_region * y_pred_region))
        self.total.assign_add(tf.reduce_sum(y_true_region) + tf.reduce_sum(y_pred_region))

    def result(self):
        dice = (2. * self.intersection + K.epsilon()) / (self.total + K.epsilon())
        return tf.where(tf.equal(self.total, 0), tf.where(tf.equal(self.intersection, 0), 1.0, 0.0), dice)

    def reset_state(self):
        self.intersection.assign(0.0)
        self.total.assign(0.0)

    def reset_states(self):
        self.reset_state()

class DiceED(tf.keras.metrics.Metric):
    def __init__(self, name='dice_ed', **kwargs):
        super(DiceED, self).__init__(name=name, **kwargs)
        self.intersection = self.add_weight(name='dice_ed_intersection', initializer='zeros')
        self.total = self.add_weight(name='dice_ed_total', initializer='zeros')
        self.built = True

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true_region = tf.cast(y_true[..., 2], tf.float32)
        y_pred_region = tf.cast(y_pred[..., 2] > 0.5, tf.float32)
        self.intersection.assign_add(tf.reduce_sum(y_true_region * y_pred_region))
        self.total.assign_add(tf.reduce_sum(y_true_region) + tf.reduce_sum(y_pred_region))

    def result(self):
        dice = (2. * self.intersection + K.epsilon()) / (self.total + K.epsilon())
        return tf.where(tf.equal(self.total, 0), tf.where(tf.equal(self.intersection, 0), 1.0, 0.0), dice)

    def reset_state(self):
        self.intersection.assign(0.0)
        self.total.assign(0.0)

    def reset_states(self):
        self.reset_state()

# --- Metrics for Derived Regions (TC, WT) from original BG,NCR,ED,ET channels ---

class DiceTC_Derived(tf.keras.metrics.Metric):
    def __init__(self, name='dice_tc_derived', **kwargs):
        super(DiceTC_Derived, self).__init__(name=name, **kwargs)
        self.intersection = self.add_weight(name='dtc_intersection', initializer='zeros')
        self.total = self.add_weight(name='dtc_total', initializer='zeros')
        self.built = True

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true_f = tf.cast(y_true, tf.float32)
        y_pred_f = tf.cast(y_pred, tf.float32)
        # TC = NCR (channel 1) or ET (channel 3)
        y_true_region = tf.cast(tf.logical_or(tf.cast(y_true_f[..., 1], tf.bool), tf.cast(y_true_f[..., 3], tf.bool)), tf.float32)
        y_pred_region = tf.cast(tf.logical_or(tf.cast(y_pred_f[..., 1] > 0.5, tf.bool), tf.cast(y_pred_f[..., 3] > 0.5, tf.bool)), tf.float32)
        self.intersection.assign_add(tf.reduce_sum(y_true_region * y_pred_region))
        self.total.assign_add(tf.reduce_sum(y_true_region) + tf.reduce_sum(y_pred_region))

    def result(self):
        dice = (2. * self.intersection + K.epsilon()) / (self.total + K.epsilon())
        return tf.where(tf.equal(self.total, 0), tf.where(tf.equal(self.intersection, 0), 1.0, 0.0), dice)

    def reset_state(self):
        self.intersection.assign(0.0)
        self.total.assign(0.0)

    def reset_states(self):
        self.reset_state()

class DiceWT_Derived(tf.keras.metrics.Metric):
    def __init__(self, name='dice_wt_derived', **kwargs):
        super(DiceWT_Derived, self).__init__(name=name, **kwargs)
        self.intersection = self.add_weight(name='dwt_intersection', initializer='zeros')
        self.total = self.add_weight(name='dwt_total', initializer='zeros')
        self.built = True

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true_f = tf.cast(y_true, tf.float32)
        y_pred_f = tf.cast(y_pred, tf.float32)
        # WT = NCR (channel 1) or ED (channel 2) or ET (channel 3)
        y_true_region = tf.cast(tf.logical_or(tf.logical_or(tf.cast(y_true_f[..., 1], tf.bool), tf.cast(y_true_f[..., 2], tf.bool)), tf.cast(y_true_f[..., 3], tf.bool)), tf.float32)
        y_pred_ncr = tf.cast(y_pred_f[..., 1] > 0.5, tf.bool)
        y_pred_ed = tf.cast(y_pred_f[..., 2] > 0.5, tf.bool)
        y_pred_et = tf.cast(y_pred_f[..., 3] > 0.5, tf.bool)
        y_pred_region = tf.cast(tf.logical_or(tf.logical_or(y_pred_ncr, y_pred_ed), y_pred_et), tf.float32)
        self.intersection.assign_add(tf.reduce_sum(y_true_region * y_pred_region))
        self.total.assign_add(tf.reduce_sum(y_true_region) + tf.reduce_sum(y_pred_region))

    def result(self):
        dice = (2. * self.intersection + K.epsilon()) / (self.total + K.epsilon())
        return tf.where(tf.equal(self.total, 0), tf.where(tf.equal(self.intersection, 0), 1.0, 0.0), dice)

    def reset_state(self):
        self.intersection.assign(0.0)
        self.total.assign(0.0)

    def reset_states(self):
        self.reset_state()

# --- Combined Dice for ET, Derived TC, Derived WT ---

class CombinedDice_Derived_TC_WT_ET(tf.keras.metrics.Metric):
    def __init__(self, name='combined_dice_derived', **kwargs):
        super(CombinedDice_Derived_TC_WT_ET, self).__init__(name=name, **kwargs)
        self.et_dice_metric = DiceET()
        self.tc_dice_derived_metric = DiceTC_Derived()
        self.wt_dice_derived_metric = DiceWT_Derived()
        self.built = True

    def update_state(self, y_true, y_pred, sample_weight=None):
        self.et_dice_metric.update_state(y_true, y_pred, sample_weight)
        self.tc_dice_derived_metric.update_state(y_true, y_pred, sample_weight)
        self.wt_dice_derived_metric.update_state(y_true, y_pred, sample_weight)

    def result(self):
        return (self.et_dice_metric.result() + self.tc_dice_derived_metric.result() + self.wt_dice_derived_metric.result()) / 3.0

    def reset_state(self):
        self.et_dice_metric.reset_state()
        self.tc_dice_derived_metric.reset_state()
        self.wt_dice_derived_metric.reset_state()

    def reset_states(self):
        self.reset_state()

In [10]:
#cell 9


import tensorflow as tf
from tensorflow.keras.layers import Conv3D, MaxPooling3D, UpSampling3D, Concatenate, Input, LeakyReLU ,Dropout, Add
from tensorflow.keras.models import Model

def nnunet_3d(input_shape=(128, 128, 128, 4), n_classes=4, base_filters=32, dropout_rate=0.2):
    inputs = Input(input_shape)

    def conv_block_with_residual(x, filters, stage, dropout_rate=0.2):
        # Store input for residual connection
        shortcut = x

        # Convolution path
        x = Conv3D(filters, 3, padding='same', use_bias=False, name=f'conv{stage}_1')(x)
        x = InstanceNormalization(name=f'in{stage}_1')(x)
        x = LeakyReLU(alpha=0.2, name=f'leaky{stage}_1')(x)  # Increased alpha

        # Add dropout for regularization
        if dropout_rate > 0:
            x = Dropout(dropout_rate, name=f'drop{stage}_1')(x)

        x = Conv3D(filters, 3, padding='same', use_bias=False, name=f'conv{stage}_2')(x)
        x = InstanceNormalization(name=f'in{stage}_2')(x)

        # Add residual connection if input has same dimensions
        if int(shortcut.shape[-1]) == filters:
            x = Add(name=f'residual{stage}')([x, shortcut])
        else:
            # If dimensions don't match, use 1x1 conv to match dimensions
            shortcut = Conv3D(filters, 1, padding='same', name=f'shortcut{stage}')(shortcut)
            x = Add(name=f'residual{stage}')([x, shortcut])

        x = LeakyReLU(alpha=0.2, name=f'leaky{stage}_out')(x)

        # Add dropout for the entire block
        if dropout_rate > 0:
            x = Dropout(dropout_rate/2, name=f'drop{stage}_out')(x)

        return x

    # Encoder with increased capacity
    c1 = conv_block_with_residual(inputs, base_filters, '1', dropout_rate)            # 128x128x128x32
    p1 = MaxPooling3D(2, name='pool1')(c1)                                            # 64x64x64x32

    c2 = conv_block_with_residual(p1, base_filters * 2, '2', dropout_rate)            # 64x64x64x64
    p2 = MaxPooling3D(2, name='pool2')(c2)                                            # 32x32x32x64

    c3 = conv_block_with_residual(p2, base_filters * 4, '3', dropout_rate)            # 32x32x32x128
    p3 = MaxPooling3D(2, name='pool3')(c3)                                            # 16x16x16x128

    c4 = conv_block_with_residual(p3, base_filters * 8, '4', dropout_rate)            # 16x16x16x256
    p4 = MaxPooling3D(2, name='pool4')(c4)                                            # 8x8x8x256

    # Bottleneck with increased capacity
    c5 = conv_block_with_residual(p4, base_filters * 16, '5', dropout_rate)           # 8x8x8x512

    # Decoder with Deep Supervision
    u4 = UpSampling3D(2, name='up4')(c5)                                              # 16x16x16x512
    u4 = Conv3D(base_filters * 8, 2, padding='same', name='upconv4')(u4)              # 16x16x16x256

    # Attention mechanism for skip connection
    att4 = tf.keras.layers.Multiply()([u4, c4])                                       # Element-wise multiplication
    concat4 = Concatenate(name='concat4')([u4, att4])                                 # Enhanced skip connection
    c6 = conv_block_with_residual(concat4, base_filters * 8, '6', dropout_rate)       # 16x16x16x256
    out4 = Conv3D(n_classes, 1, activation='softmax', name='out4')(c6)

    u3 = UpSampling3D(2, name='up3')(c6)                                              # 32x32x32x256
    u3 = Conv3D(base_filters * 4, 2, padding='same', name='upconv3')(u3)              # 32x32x32x128

    # Attention for skip connection
    att3 = tf.keras.layers.Multiply()([u3, c3])                                       # Element-wise multiplication
    concat3 = Concatenate(name='concat3')([u3, att3])                                 # Enhanced skip connection
    c7 = conv_block_with_residual(concat3, base_filters * 4, '7', dropout_rate)       # 32x32x32x128
    out3 = Conv3D(n_classes, 1, activation='softmax', name='out3')(c7)

    u2 = UpSampling3D(2, name='up2')(c7)                                              # 64x64x64x128
    u2 = Conv3D(base_filters * 2, 2, padding='same', name='upconv2')(u2)              # 64x64x64x64

    # Attention for skip connection
    att2 = tf.keras.layers.Multiply()([u2, c2])                                       # Element-wise multiplication
    concat2 = Concatenate(name='concat2')([u2, att2])                                 # Enhanced skip connection
    c8 = conv_block_with_residual(concat2, base_filters * 2, '8', dropout_rate)       # 64x64x64x64
    out2 = Conv3D(n_classes, 1, activation='softmax', name='out2')(c8)

    u1 = UpSampling3D(2, name='up1')(c8)                                              # 128x128x128x64
    u1 = Conv3D(base_filters, 2, padding='same', name='upconv1')(u1)                  # 128x128x128x32

    # Attention for skip connection
    att1 = tf.keras.layers.Multiply()([u1, c1])                                       # Element-wise multiplication
    concat1 = Concatenate(name='concat1')([u1, att1])                                 # Enhanced skip connection
    c9 = conv_block_with_residual(concat1, base_filters, '9', dropout_rate/2)         # 128x128x128x32
    outputs = Conv3D(n_classes, 1, activation='softmax', name='out_final')(c9)

    return Model(inputs=inputs, outputs=[outputs, out2, out3, out4])

In [11]:
#cell 10
batch_size = 2

train_dataset = tf.data.Dataset.from_generator(
    lambda: data_generator(train_patients, train_data_path, is_training=True),
    output_signature=(
        tf.TensorSpec(shape=(128, 128, 128, 4), dtype=tf.float32),
        (
            tf.TensorSpec(shape=(128, 128, 128, 4), dtype=tf.float32),  # out_final
            tf.TensorSpec(shape=(64, 64, 64, 4), dtype=tf.float32),     # out2
            tf.TensorSpec(shape=(32, 32, 32, 4), dtype=tf.float32),     # out3
            tf.TensorSpec(shape=(16, 16, 16, 4), dtype=tf.float32)      # out4
        )
    )
).shuffle(buffer_size=10).batch(batch_size).repeat().prefetch(tf.data.AUTOTUNE)

val_dataset = tf.data.Dataset.from_generator(
    lambda: data_generator(val_patients, train_data_path, is_training=False),
    output_signature=(
        tf.TensorSpec(shape=(128, 128, 128, 4), dtype=tf.float32),
        (
            tf.TensorSpec(shape=(128, 128, 128, 4), dtype=tf.float32),  # out_final
            tf.TensorSpec(shape=(64, 64, 64, 4), dtype=tf.float32),     # out2
            tf.TensorSpec(shape=(32, 32, 32, 4), dtype=tf.float32),     # out3
            tf.TensorSpec(shape=(16, 16, 16, 4), dtype=tf.float32)      # out4
        )
    )
).batch(batch_size).repeat().prefetch(tf.data.AUTOTUNE)

In [ ]:
#cell 10
import tensorflow as tf
import os

'''
# Improved callbacks
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor="val_out_final_combined_dice",
        patience=15,  # Increased patience
        restore_best_weights=True,
        mode="max",
        verbose=1
    ),
    tf.keras.callbacks.TensorBoard(
        log_dir=os.path.join(output_dir, "logs"),
        histogram_freq=1,
        write_graph=True,
        write_images=True
    ),
    # Add a custom callback to save and evaluate predictions
    tf.keras.callbacks.CSVLogger(
        os.path.join(output_dir, "training_log.csv"),
        append=True
    )
]
'''

'\n# Improved callbacks\ncallbacks = [\n    tf.keras.callbacks.EarlyStopping(\n        monitor="val_out_final_combined_dice",\n        patience=15,  # Increased patience\n        restore_best_weights=True,\n        mode="max",\n        verbose=1\n    ),\n    tf.keras.callbacks.TensorBoard(\n        log_dir=os.path.join(output_dir, "logs"),\n        histogram_freq=1,\n        write_graph=True,\n        write_images=True\n    ),\n    # Add a custom callback to save and evaluate predictions\n    tf.keras.callbacks.CSVLogger(\n        os.path.join(output_dir, "training_log.csv"),\n        append=True\n    )\n]\n'

In [ ]:
#cell 11
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, Callback, EarlyStopping, TensorBoard
from tensorflow.keras import backend as K
import os
import re
import pandas as pd

# Prerequisites:
# - Metric classes from "Updated Cell 7" (above) must be defined/executed.
# - 'nnunet_3d' model function must be defined (your original Cell 8).
# - 'train_patients', 'val_patients', 'batch_size' must be defined.
# - 'train_dataset', 'val_dataset' (tf.data.Dataset objects) must be defined.
#   (Ensure data_generator -> preprocess_segmentation_mask outputs one-hot original labels).

#print("Available GPUs:", tf.config.list_physical_devices('GPU'))
# The following line might error if no GPU is configured or if TF is not a GPU version.
# You can comment it out if it causes issues:
print("GPU memory:", tf.config.experimental.get_memory_info('GPU:0'))


# --- Configuration for this Training Run ---
# Define a base directory for this experiment's outputs
# This helps keep different runs organized.
BRATS_CHALLENGE_BASE_OUTPUT_DIR = "/content/drive/MyDrive/Brats_Challenge/model_output" # Your base model_output dir
EXPERIMENT_NAME = "run_with_derived_metrics_v1" # Change this for different experiments

# Specific paths for this experiment
CURRENT_EXPERIMENT_DIR = os.path.join(BRATS_CHALLENGE_BASE_OUTPUT_DIR, EXPERIMENT_NAME)
CHECKPOINT_SAVE_DIR = os.path.join(CURRENT_EXPERIMENT_DIR, "checkpoints") # Checkpoints within experiment folder
METRICS_LOG_FILE = os.path.join(CURRENT_EXPERIMENT_DIR, f"training_log_{EXPERIMENT_NAME}.csv")
BEST_WEIGHTS_FILEPATH = os.path.join(CHECKPOINT_SAVE_DIR, f"best_model_{EXPERIMENT_NAME}.weights.h5")
TENSORBOARD_LOG_DIR = os.path.join(CURRENT_EXPERIMENT_DIR, f"logs_{EXPERIMENT_NAME}")

os.makedirs(CHECKPOINT_SAVE_DIR, exist_ok=True)
os.makedirs(TENSORBOARD_LOG_DIR, exist_ok=True)

print(f"Outputs for this run (checkpoints, logs) will be in: {CURRENT_EXPERIMENT_DIR}")


# --- Callbacks ---
model_checkpoint_derived = ModelCheckpoint(
    filepath=BEST_WEIGHTS_FILEPATH,
    save_best_only=True,
    monitor="val_out_final_combined_dice_derived", # Monitor the new combined Dice
    mode="max",
    save_weights_only=True,
    verbose=1
)

class CustomMetricsLoggerDerived(Callback):
    def __init__(self, metrics_file_path):
        super(CustomMetricsLoggerDerived, self).__init__()
        self.metrics_file_path = metrics_file_path

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        metrics_to_log = {
            'epoch': epoch + 1, 'loss': logs.get('loss'), 'val_loss': logs.get('val_loss'),
            'val_dice_et': logs.get('val_out_final_dice_et'),
            'val_dice_tc_derived': logs.get('val_out_final_dice_tc_derived'),
            'val_dice_wt_derived': logs.get('val_out_final_dice_wt_derived'),
            'val_combined_dice_derived': logs.get('val_out_final_combined_dice_derived'),
            'val_dice_ncr': logs.get('val_out_final_dice_ncr'), # Logging original components
            'val_dice_ed': logs.get('val_out_final_dice_ed'),   # Logging original components
        }
        print(f"\nEpoch {epoch + 1} Validation Metrics:")
        for k, v in metrics_to_log.items():
            if v is not None: print(f"  {k}: {v:.4f}" if isinstance(v, float) else f"  {k}: {v}")

        df = pd.DataFrame([metrics_to_log])
        header = not os.path.exists(self.metrics_file_path) or os.path.getsize(self.metrics_file_path) == 0
        try:
            df.to_csv(self.metrics_file_path, mode='a', header=header, index=False)
        except Exception as e:
            print(f"Error writing to CSV {self.metrics_file_path}: {e}")

early_stopping_derived = EarlyStopping(
    monitor="val_out_final_combined_dice_derived",
    patience=25, # Consider adjusting patience
    restore_best_weights=True,
    mode="max",
    verbose=1
)
tensorboard_callback_derived = TensorBoard(log_dir=TENSORBOARD_LOG_DIR, histogram_freq=1)

all_training_callbacks = [
    model_checkpoint_derived,
    CustomMetricsLoggerDerived(METRICS_LOG_FILE),
    early_stopping_derived,
    tensorboard_callback_derived
]

# --- Loss Function (combo_loss from your original code) ---
# This loss function operates on the one-hot encoded y_true [BG, NCR, ED, ET]
# and the softmax y_pred from the model.
@tf.function
def combo_loss(y_true_list, y_pred_list, smooth=1e-6, alpha=0.5, beta=1.0, gamma_focal=1.5, tversky_coeff=0.4):
    # Class weights for [BG, NCR, ED, ET] as per your setup that achieved good results
    class_weights = tf.constant([0.1, 6.2, 2.5, 6.0], dtype=tf.float32)

    def weighted_focal_categorical_crossentropy(y_true, y_pred):
        y_pred = tf.clip_by_value(y_pred, 1e-7, 1.0 - 1e-7)
        focal_weight = tf.pow(1.0 - y_pred, gamma_focal)
        crossentropy = -tf.reduce_sum(y_true * focal_weight * tf.math.log(y_pred) * class_weights, axis=-1)
        return tf.reduce_mean(crossentropy)

    def dice_loss_internal(y_true, y_pred):
        numerator = 0.0
        denominator = 0.0
        for c in range(1, 4):
            class_true = y_true[..., c]
            class_pred = y_pred[..., c]
            focal_weight = tf.pow(1.0 - tf.reduce_mean(class_pred), gamma_focal / 2.0)
            intersection = tf.reduce_sum(class_true * class_pred) * focal_weight
            union = tf.reduce_sum(class_true) + tf.reduce_sum(class_pred)
            numerator += intersection * class_weights[c]
            denominator += union * class_weights[c]
        dice = (2.0 * numerator + smooth) / (denominator + smooth)
        return 1.0 - dice

    def tversky_loss_internal(y_true, y_pred, alpha_tv=0.7, beta_tv=0.3):
        numerator = 0.0
        denominator = 0.0
        for c in range(1, 4):
            class_true = y_true[..., c]
            class_pred = y_pred[..., c]
            true_pos = tf.reduce_sum(class_true * class_pred)
            false_neg = tf.reduce_sum(class_true * (1 - class_pred))
            false_pos = tf.reduce_sum((1 - class_true) * class_pred)
            tversky = (true_pos + smooth) / (true_pos + alpha_tv * false_neg + beta_tv * false_pos + smooth)
            numerator += tversky * class_weights[c]
            denominator += class_weights[c]
        tversky = numerator / denominator
        return 1.0 - tversky

        # Average of weighted Tversky indices
        average_weighted_tversky = 1.0 # Default to max loss if no relevant classes considered
        if sum_of_class_weights_for_norm > 0:
             average_weighted_tversky = weighted_tversky_sum / sum_of_class_weights_for_norm
        return 1.0 - average_weighted_tversky

    total_loss = tf.constant(0.0, dtype=tf.float32)
    for i in tf.range(len(y_true_list)):
        y_true = y_true_list[i]
        y_pred = y_pred_list[i]
        if y_pred.shape[1:-1] != y_true.shape[1:-1]:
            y_pred = tf.image.resize(y_pred, y_true.shape[1:-1], method='nearest')
        ce_component = weighted_focal_categorical_crossentropy(y_true, y_pred)
        dice_component = dice_loss_internal(y_true, y_pred)
        tversky_component = tversky_loss_internal(y_true, y_pred, alpha_tv=0.7, beta_tv=0.3)
        combined = alpha * ce_component + beta * dice_component + 0.4 * tversky_component
        total_loss += combined
    return total_loss / tf.cast(len(y_true_list), tf.float32)


# --- Model Build, Compile, Train ---
# Ensure 'nnunet_3d' model function is defined (from your original Cell 8)
# Example: model = nnunet_3d(input_shape=(128, 128, 128, 4), n_classes=4, base_filters=32, dropout_rate=0.2)
# ^^^ This line above for 'model =' needs to be your actual model instantiation.
# If 'model' is already defined and compiled from a previous cell, you might not need to redefine or recompile,
# but if you are changing metrics, recompilation is necessary.
# Build the model
model = nnunet_3d(input_shape=(128, 128, 128, 4), n_classes=4, base_filters=32, dropout_rate=0.2)
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5), # Or AdamW(learning_rate=5e-5, weight_decay=1e-5)
    loss=combo_loss,
    metrics={
        # Metrics for the main output head ('out_final')
        'out_final': ['accuracy', DiceET(), DiceTC_Derived(), DiceWT_Derived(), CombinedDice_Derived_TC_WT_ET(), DiceNCR(), DiceED()],
        # Metrics for deep supervision output heads (names 'out2', 'out3', 'out4' must match your model's output names)
        'out2': ['accuracy', DiceET(), DiceTC_Derived(), DiceWT_Derived(), DiceNCR(), DiceED()],
        'out3': ['accuracy', DiceET(), DiceTC_Derived(), DiceWT_Derived(), DiceNCR(), DiceED()],
        'out4': ['accuracy', DiceET(), DiceTC_Derived(), DiceWT_Derived(), DiceNCR(), DiceED()]
    }
)
model.summary() # Good to check model structure and output names

initial_epoch = 0
if os.path.exists(BEST_WEIGHTS_FILEPATH):
    print(f"Loading best weights from: {BEST_WEIGHTS_FILEPATH}")
    model.load_weights(BEST_WEIGHTS_FILEPATH)
    # Add logic here to parse 'initial_epoch' from filename if your checkpoint names contain epoch info
    # and you want to resume precisely. For now, if weights are loaded, epoch starts from 0 for new fit,
    # or you can manually set 'initial_epoch'.
else:
    print(f"No weights found at {BEST_WEIGHTS_FILEPATH}. Starting fresh or using other preloaded weights.")

# Ensure these variables are correctly defined from your data setup cells:
# train_patients, val_patients, batch_size (used to calculate steps)
# train_dataset, val_dataset (your tf.data.Dataset objects)
train_steps = max(1, len(train_patients) // batch_size)
val_steps = max(1, len(val_patients) // batch_size)

print(f"Calculated train_steps per epoch: {train_steps}")
print(f"Calculated validation_steps per epoch: {val_steps}")

print("Verifying 1 sample from train_dataset for shapes:")
for x_train_batch, y_train_batch in train_dataset.take(1):
    print(f"  Train input batch shape: {x_train_batch.shape}")
    if isinstance(y_train_batch, tuple): # For deep supervision
        print(f"  Train target batch shapes (y_true_list items): {[y_i.shape for y_i in y_train_batch]}")
    else:
        print(f"  Train target batch shape: {y_train_batch.shape}")

print("Verifying 1 sample from val_dataset for shapes:")
for x_val_batch, y_val_batch in val_dataset.take(1):
    print(f"  Val input batch shape: {x_val_batch.shape}")
    if isinstance(y_val_batch, tuple): # For deep supervision
        print(f"  Val target batch shapes (y_true_list items): {[y_i.shape for y_i in y_val_batch]}")
    else:
        print(f"  Val target batch shape: {y_val_batch.shape}")

print("Starting model training...")
history = model.fit(
    train_dataset,
    epochs=15, # Set your desired total number of epochs for this run
    initial_epoch=initial_epoch, # Set to last completed epoch if resuming
    steps_per_epoch=train_steps,
    validation_data=val_dataset,
    validation_steps=val_steps,
    callbacks=all_training_callbacks,
    verbose=2
)
print("Model training finished.")

GPU memory: {'current': 439898880, 'peak': 13917301760}
Outputs for this run (checkpoints, logs) will be in: /content/drive/MyDrive/Brats_Challenge/model_output/run_with_derived_metrics_v1


Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_7       │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 128, 4)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_1 (Conv3D)    │ (None, 128, 128,  │      3,456 │ input_layer_7[0]… │
│                     │ 128, 32)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ in1_1               │ (None, 128, 128,  │         64 │ conv1_1[0][0]     │
│ (InstanceNormaliza… │ 128, 32)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky1_1            │ (None, 128, 128,  │          0 │ in1_1[0][0]       │
│ (LeakyReLU)         │ 128, 32)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ drop1_1 (Dropout)   │ (None, 128, 128,  │          0 │ leaky1_1[0][0]    │
│                     │ 128, 32)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_2 (Conv3D)    │ (None, 128, 128,  │     27,648 │ drop1_1[0][0]     │
│                     │ 128, 32)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ in1_2               │ (None, 128, 128,  │         64 │ conv1_2[0][0]     │
│ (InstanceNormaliza… │ 128, 32)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ shortcut1 (Conv3D)  │ (None, 128, 128,  │        160 │ input_layer_7[0]… │
│                     │ 128, 32)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ residual1 (Add)     │ (None, 128, 128,  │          0 │ in1_2[0][0],      │
│                     │ 128, 32)          │            │ shortcut1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky1_out          │ (None, 128, 128,  │          0 │ residual1[0][0]   │
│ (LeakyReLU)         │ 128, 32)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ drop1_out (Dropout) │ (None, 128, 128,  │          0 │ leaky1_out[0][0]  │
│                     │ 128, 32)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1               │ (None, 64, 64,    │          0 │ drop1_out[0][0]   │
│ (MaxPooling3D)      │ 64, 32)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_1 (Conv3D)    │ (None, 64, 64,    │     55,296 │ pool1[0][0]       │
│                     │ 64, 64)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ in2_1               │ (None, 64, 64,    │        128 │ conv2_1[0][0]     │
│ (InstanceNormaliza… │ 64, 64)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky2_1            │ (None, 64, 64,    │          0 │ in2_1[0][0]       │
│ (LeakyReLU)         │ 64, 64)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ drop2_1 (Dropout)   │ (None, 64, 64,    │          0 │ leaky2_1[0][0]    │
│                     │ 64, 64)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_2 (Conv3D)    │ (None, 64, 64,    │    110,592 │ drop2_1[0][0]   

 Total params: 22,932,528 (87.48 MB)

 Trainable params: 22,932,528 (87.48 MB)

 Non-trainable params: 0 (0.00 B)

No weights found at /content/drive/MyDrive/Brats_Challenge/model_output/run_with_derived_metrics_v1/checkpoints/best_model_run_with_derived_metrics_v1.weights.h5. Starting fresh or using other preloaded weights.
Calculated train_steps per epoch: 472
Calculated validation_steps per epoch: 101
Verifying 1 sample from train_dataset for shapes:
Step 1/945: Processing BraTS-GoAT-01970
  Train input batch shape: (2, 128, 128, 128, 4)
  Train target batch shapes (y_true_list items): [TensorShape([2, 128, 128, 128, 4]), TensorShape([2, 64, 64, 64, 4]), TensorShape([2, 32, 32, 32, 4]), TensorShape([2, 16, 16, 16, 4])]
Verifying 1 sample from val_dataset for shapes:
Step 1/203: Processing BraTS-GoAT-02355
  Val input batch shape: (2, 128, 128, 128, 4)
  Val target batch shapes (y_true_list items): [TensorShape([2, 128, 128, 128, 4]), TensorShape([2, 64, 64, 64, 4]), TensorShape([2, 32, 32, 32, 4]), TensorShape([2, 16, 16, 16, 4])]
Starting model training...
Epoch 1/15
Step 1/945: Processing BraT

In [ ]:

#cell 12
import pandas as pd
import matplotlib.pyplot as plt
import os

# These variables MUST be defined and consistent with your Cell 11 training script:
BRATS_CHALLENGE_BASE_OUTPUT_DIR = "/content/drive/MyDrive/Brats_Challenge/model_output"
EXPERIMENT_NAME = "run_with_derived_metrics_v1" # Or your current experiment name
CURRENT_EXPERIMENT_DIR = os.path.join(BRATS_CHALLENGE_BASE_OUTPUT_DIR, EXPERIMENT_NAME)
METRICS_LOG_FILE = os.path.join(CURRENT_EXPERIMENT_DIR, f"training_log_{EXPERIMENT_NAME}.csv")

# Define a directory for saving plots within your current experiment directory
PLOTS_SAVE_DIR = os.path.join(CURRENT_EXPERIMENT_DIR, "plots_from_training_log")
os.makedirs(PLOTS_SAVE_DIR, exist_ok=True)

if os.path.exists(METRICS_LOG_FILE):
    try:
        df = pd.read_csv(METRICS_LOG_FILE)

        if df.empty:
            print(f"Metrics file {METRICS_LOG_FILE} is empty. No plots will be generated.")
        else:
            # Plot 1: Validation Loss
            if 'epoch' in df.columns and 'val_loss' in df.columns:
                plt.figure(figsize=(10, 6))
                plt.plot(df['epoch'], df['val_loss'], 'b-', label='Validation Loss', marker='o', markersize=3)
                plt.title('Validation Loss Over Epochs')
                plt.xlabel('Epoch')
                plt.ylabel('Loss')
                plt.grid(True)
                plt.legend()
                plot_path = os.path.join(PLOTS_SAVE_DIR, 'validation_loss_plot.png')
                plt.savefig(plot_path)
                plt.close()
                print(f"Validation loss plot saved to: {plot_path}")
            else:
                print("Could not plot validation loss: 'epoch' or 'val_loss' column missing in CSV.")

            # Plot 2: Dice Scores
            if 'epoch' in df.columns:
                plt.figure(figsize=(12, 8))
                plot_dice_scores_available = False

                # These column names match the keys in 'metrics_to_log' from your CustomMetricsLoggerDerived
                if 'val_dice_et' in df.columns:
                    plt.plot(df['epoch'], df['val_dice_et'], label='ET Dice', linestyle='-', marker='o', markersize=4)
                    plot_dice_scores_available = True
                if 'val_dice_tc_derived' in df.columns:
                    plt.plot(df['epoch'], df['val_dice_tc_derived'], label='TC Dice (Derived)', linestyle='-', marker='s', markersize=4)
                    plot_dice_scores_available = True
                if 'val_dice_wt_derived' in df.columns:
                    plt.plot(df['epoch'], df['val_dice_wt_derived'], label='WT Dice (Derived)', linestyle='-', marker='^', markersize=4)
                    plot_dice_scores_available = True
                if 'val_combined_dice_derived' in df.columns:
                    plt.plot(df['epoch'], df['val_combined_dice_derived'], label='Combined Dice (ET,TC,WT)', linestyle='--', marker='d', markersize=4)
                    plot_dice_scores_available = True

                # Optional: Plotting NCR and ED Dice if they are in the CSV
                if 'val_dice_ncr' in df.columns:
                    plt.plot(df['epoch'], df['val_dice_ncr'], label='NCR Dice', linestyle=':', marker='x', markersize=4)
                    plot_dice_scores_available = True
                if 'val_dice_ed' in df.columns:
                    plt.plot(df['epoch'], df['val_dice_ed'], label='ED Dice', linestyle=':', marker='+', markersize=4)
                    plot_dice_scores_available = True

                if plot_dice_scores_available:
                    plt.title('Validation Dice Scores Over Epochs')
                    plt.xlabel('Epoch')
                    plt.ylabel('Dice Score')
                    # Attempt to set a reasonable Y-axis limit for Dice scores
                    min_dice_val = 0.0
                    if any(col in df.columns for col in ['val_dice_et', 'val_dice_tc_derived', 'val_dice_wt_derived', 'val_combined_dice_derived', 'val_dice_ncr', 'val_dice_ed']):
                        dice_cols_present = [col for col in ['val_dice_et', 'val_dice_tc_derived', 'val_dice_wt_derived', 'val_combined_dice_derived', 'val_dice_ncr', 'val_dice_ed'] if col in df.columns]
                        if df[dice_cols_present].notna().any().any(): # Check if there's any non-NaN data
                             min_dice_val = df[dice_cols_present].min(skipna=True).min(skipna=True)
                    plt.ylim(max(0, min_dice_val - 0.05), 1.05)
                    plt.grid(True)
                    plt.legend(loc='best')
                    plt.tight_layout()
                    plot_path = os.path.join(PLOTS_SAVE_DIR, 'dice_scores_plot.png')
                    plt.savefig(plot_path)
                    plt.close()
                    print(f"Dice scores plot saved to: {plot_path}")
                else:
                    print("No relevant Dice score columns found in CSV for plotting.")
            else:
                print("Could not plot Dice scores: 'epoch' column missing in CSV.")
    except pd.errors.EmptyDataError:
        print(f"Metrics file {METRICS_LOG_FILE} is empty. No plots will be generated.")
    except Exception as e:
        print(f"An error occurred while reading or plotting {METRICS_LOG_FILE}: {e}")
else:
    print(f"Metrics file {METRICS_LOG_FILE} not found. Cannot generate plots.")

Metrics file /content/drive/MyDrive/Brats_Challenge/model_output/run_with_derived_metrics_v1/training_log_run_with_derived_metrics_v1.csv not found. Cannot generate plots.


In [14]:
#cell 13

import tensorflow as tf
import numpy as np
import os
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

def calculate_dice_et_from_onehot(y_true_one_hot, y_pred_final_one_hot, smooth=1e-6):
    y_true_region = y_true_one_hot[..., 3].astype(np.float32)
    y_pred_region = y_pred_final_one_hot[..., 3].astype(np.float32)
    intersection = np.sum(y_true_region * y_pred_region)
    sum_true_plus_pred = np.sum(y_true_region) + np.sum(y_pred_region)
    dice = (2. * intersection + smooth) / (sum_true_plus_pred + smooth)
    if sum_true_plus_pred == 0 and intersection == 0:
        dice = 1.0
    return dice

def calculate_dice_tc_from_onehot(y_true_one_hot, y_pred_final_one_hot, smooth=1e-6):
    y_true_ncr = y_true_one_hot[..., 1]
    y_true_et = y_true_one_hot[..., 3]
    y_true_region = np.logical_or(y_true_ncr, y_true_et).astype(np.float32)
    y_pred_ncr = y_pred_final_one_hot[..., 1]
    y_pred_et = y_pred_final_one_hot[..., 3]
    y_pred_region = np.logical_or(y_pred_ncr, y_pred_et).astype(np.float32)
    intersection = np.sum(y_true_region * y_pred_region)
    sum_true_plus_pred = np.sum(y_true_region) + np.sum(y_pred_region)
    dice = (2. * intersection + smooth) / (sum_true_plus_pred + smooth)
    if sum_true_plus_pred == 0 and intersection == 0:
        dice = 1.0
    return dice

def calculate_dice_wt_from_onehot(y_true_one_hot, y_pred_final_one_hot, smooth=1e-6):
    y_true_ncr = y_true_one_hot[..., 1]
    y_true_ed = y_true_one_hot[..., 2]
    y_true_et = y_true_one_hot[..., 3]
    y_true_region = np.logical_or(np.logical_or(y_true_ncr, y_true_ed), y_true_et).astype(np.float32)
    y_pred_ncr = y_pred_final_one_hot[..., 1]
    y_pred_ed = y_pred_final_one_hot[..., 2]
    y_pred_et = y_pred_final_one_hot[..., 3]
    y_pred_region = np.logical_or(np.logical_or(y_pred_ncr, y_pred_ed), y_pred_et).astype(np.float32)
    intersection = np.sum(y_true_region * y_pred_region)
    sum_true_plus_pred = np.sum(y_true_region) + np.sum(y_pred_region)
    dice = (2. * intersection + smooth) / (sum_true_plus_pred + smooth)
    if sum_true_plus_pred == 0 and intersection == 0:
        dice = 1.0
    return dice

BRATS_CHALLENGE_BASE_OUTPUT_DIR = "/content/drive/MyDrive/Brats_Challenge/model_output"
EXPERIMENT_NAME = "run_with_derived_metrics_v1"
BEST_WEIGHTS_FILEPATH = os.path.join(BRATS_CHALLENGE_BASE_OUTPUT_DIR, EXPERIMENT_NAME, "checkpoints", f"best_model_{EXPERIMENT_NAME}.weights.h5")
CURRENT_EXPERIMENT_DIR = os.path.join(BRATS_CHALLENGE_BASE_OUTPUT_DIR, EXPERIMENT_NAME)

model_test = nnunet_3d(input_shape=(128, 128, 128, 4), n_classes=4, base_filters=32, dropout_rate=0.2)

if os.path.exists(BEST_WEIGHTS_FILEPATH):
    print(f"Loading saved best weights from: {BEST_WEIGHTS_FILEPATH}")
    model_test.load_weights(BEST_WEIGHTS_FILEPATH)
else:
    print(f"ERROR: Weights file not found at {BEST_WEIGHTS_FILEPATH}.")
    raise FileNotFoundError(f"Model weights not found: {BEST_WEIGHTS_FILEPATH}")

print("Model architecture created and best weights loaded for testing.")

test_batch_size = 1

def test_data_generator_wrapper():
    for image_tensor, seg_tensors_tuple in data_generator(test_patients, train_data_path, is_training=False, target_shape=(128, 128, 128)):
        yield image_tensor, seg_tensors_tuple[0]

test_dataset_tf = tf.data.Dataset.from_generator(
    test_data_generator_wrapper,
    output_signature=(
        tf.TensorSpec(shape=(128, 128, 128, 4), dtype=tf.float32),
        tf.TensorSpec(shape=(128, 128, 128, 4), dtype=tf.float32)
    )
).batch(test_batch_size)

all_dice_scores_et = []
all_dice_scores_tc = []
all_dice_scores_wt = []

print(f"\nStarting evaluation on {len(test_patients)} test patients...")
if not test_patients:
    print("Warning: test_patients list is empty. No evaluation will be performed.")

for step, (x_batch_test, y_true_one_hot_batch) in enumerate(test_dataset_tf):
    # The test dataset yields batches of (image, main_segmentation_mask)
    print(f"Processing test sample {step + 1}/{len(test_patients)}...")

    # model_test.predict_on_batch returns a list of predictions, one for each output head.
    # The first element in the list corresponds to the 'out_final' head (128x128x128x4).
    y_pred_probs_list = model_test.predict_on_batch(x_batch_test)
    y_pred_probs_batch = y_pred_probs_list[0] # Get the predictions from the main output head

    # Now y_pred_probs_batch is a tensor/numpy array from the main output,
    # and you can access its shape to iterate over the batch size.
    for i in range(y_pred_probs_batch.shape[0]):
        y_pred_probs_single = y_pred_probs_batch[i, ...]
        y_true_one_hot_single = y_true_one_hot_batch[i, ...]

        # Convert probabilities to predicted class labels (argmax along the channel axis)
        pred_labels_int = np.argmax(y_pred_probs_single, axis=-1)

        # Convert predicted class labels back to one-hot encoding
        y_pred_final_one_hot = tf.keras.utils.to_categorical(pred_labels_int, num_classes=4)
        y_pred_final_one_hot = y_pred_final_one_hot.astype(np.float32)

        # Ensure y_true is also a numpy array of the correct type
        y_true_one_hot_single_np = y_true_one_hot_single.numpy().astype(np.float32)

        # Calculate Dice scores using the one-hot encoded true and predicted masks
        # Ensure calculate_dice functions expect numpy arrays, not tensors
        dice_et = calculate_dice_et_from_onehot(y_true_one_hot_single_np, y_pred_final_one_hot)
        dice_tc = calculate_dice_tc_from_onehot(y_true_one_hot_single_np, y_pred_final_one_hot)
        dice_wt = calculate_dice_wt_from_onehot(y_true_one_hot_single_np, y_pred_final_one_hot)

        all_dice_scores_et.append(dice_et)
        all_dice_scores_tc.append(dice_tc)
        all_dice_scores_wt.append(dice_wt)

if all_dice_scores_et:
    avg_dice_et = np.mean(all_dice_scores_et)
    avg_dice_tc = np.mean(all_dice_scores_tc)
    avg_dice_wt = np.mean(all_dice_scores_wt)

    print("\n--- Average Test Dice Scores ---")
    print(f"Enhancing Tumor (ET): {avg_dice_et:.4f} (from {len(all_dice_scores_et)} samples)")
    print(f"Tumor Core (TC) Derived: {avg_dice_tc:.4f} (from {len(all_dice_scores_tc)} samples)")
    print(f"Whole Tumor (WT) Derived: {avg_dice_wt:.4f} (from {len(all_dice_scores_wt)} samples)")

    overall_mean_dice = np.mean([avg_dice_et, avg_dice_tc, avg_dice_wt])
    print(f"Mean of (ET, TC, WT) Dice: {overall_mean_dice:.4f}")

    test_results_df = pd.DataFrame({
        'PatientID': test_patients[:len(all_dice_scores_et)],
        'Dice_ET': all_dice_scores_et,
        'Dice_TC': all_dice_scores_tc,
        'Dice_WT': all_dice_scores_wt
    })
    results_csv_path = os.path.join(CURRENT_EXPERIMENT_DIR, f"test_scores_{EXPERIMENT_NAME}.csv")
    test_results_df.to_csv(results_csv_path, index=False)
    print(f"Detailed test scores saved to: {results_csv_path}")
else:
    print("No test scores were calculated. Check if 'test_patients' list is populated and if the test loop ran.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Loading saved best weights from: /content/drive/MyDrive/Brats_Challenge/model_output/run_with_derived_metrics_v1/checkpoints/best_model_run_with_derived_metrics_v1.weights.h5
Model architecture created and best weights loaded for testing.

Starting evaluation on 203 test patients...
Step 1/203: Processing BraTS-GoAT-00679
Processing test sample 1/203...
Processing test sample 2/203...
Processing test sample 3/203...
Processing test sample 4/203...
Processing test sample 5/203...
Processing test sample 6/203...
Processing test sample 7/203...
Processing test sample 8/203...
Processing test sample 9/203...
Processing test sample 10/203...
Processing test sample 11/203...
Processing test sample 12/203...
Processing test sample 13/203...
Processing test sample 14/203...
Processing test sample 15/203...
Processing test sample 16/203...
Processing test sample 17/20

In [15]:
#cell 14

import nibabel as nib

def calculate_volumes(seg_mask_data, affine):
    """
    Calculates the volume of tumor regions in cm³.

    Args:
        seg_mask_data (np.array): The 3D numpy array of the predicted segmentation.
        affine (np.array): The affine matrix from the original NIfTI file.

    Returns:
        dict: A dictionary containing the volumes of ET, TC, and WT in cm³.
    """
    # Calculate the volume of a single voxel in mm³
    voxel_volume_mm3 = np.abs(np.linalg.det(affine[:3, :3]))

    # Convert voxel volume to cm³ (1 cm³ = 1000 mm³)
    voxel_volume_cm3 = voxel_volume_mm3 / 1000.0

    # Count the number of voxels for each class
    # Class 0: Background, 1: Necrotic Core (NCR), 2: Edema (ED), 3: Enhancing Tumor (ET)
    ncr_voxels = np.sum(seg_mask_data == 1)
    ed_voxels = np.sum(seg_mask_data == 2)
    et_voxels = np.sum(seg_mask_data == 3)

    # Calculate volumes based on definitions
    et_volume = et_voxels * voxel_volume_cm3
    tc_volume = (ncr_voxels + et_voxels) * voxel_volume_cm3 # Tumor Core = NCR + ET
    wt_volume = (ncr_voxels + et_voxels + ed_voxels) * voxel_volume_cm3 # Whole Tumor = NCR + ET + ED

    return {'ET': et_volume, 'TC': tc_volume, 'WT': wt_volume}

In [16]:
#cell 15

!pip install -q google-generativeai
import google.generativeai as genai
from google.colab import userdata

# 1. Get Your Gemini API Key
# Go to Google AI Studio and create an API key.
# In Colab, click the key icon (🔑) on the left, add a new secret,
# and name it 'GEMINI_API_KEY'. Paste your key as the value.

try:
    GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
    genai.configure(api_key=GEMINI_API_KEY)
except Exception as e:
    print(f"API Key not found in Colab Secrets. Please add it. Error: {e}")

# 2. Define the Report Generation Function
def generate_radiology_report(tumor_volumes):
    """
    Generates a descriptive report using Gemini based on segmentation volumes.
    """
    if not GEMINI_API_KEY:
        return "Report generation is unavailable because the Gemini API key is not configured."

    model = genai.GenerativeModel('gemini-1.5-flash')
    prompt = f"""
    You are an assistant aiding a radiologist. Based on AI-powered 3D segmentation, generate a structured, descriptive report.

    **Quantitative Analysis Data:**
    - Estimated Tumor Volumes:
      - Enhancing Tumor (ET): {tumor_volumes['ET']:.2f} cm³
      - Tumor Core (TC): {tumor_volumes['TC']:.2f} cm³
      - Whole Tumor (WT): {tumor_volumes['WT']:.2f} cm³

    **Instructions:**
    1.  Start with "Automated Brain Tumor Segmentation Report".
    2.  Create a "Summary" section interpreting the findings.
    3.  Create a "Quantitative Results" section with the volumes.
    4.  Add a "Disclaimer" that this is AI-generated and requires professional verification.
    5.  Maintain a professional, clinical tone. Do NOT provide a diagnosis.
    """
    response = model.generate_content(prompt)
    return response.text

In [1]:
#cell 17

import numpy as np
import nibabel as nib
import tensorflow as tf

# --- PREREQUISITES ---
# This script assumes that the following functions are already defined in a previous cell:
# - nnunet_3d
# - InstanceNormalization
# - preprocess_sample
# - calculate_volumes
# - generate_radiology_report

# --- 1. DEFINE FILE PATHS (‼️ YOU MUST CHANGE THESE ‼️) ---
MODEL_WEIGHTS_PATH = '/content/drive/MyDrive/Brats_Challenge/model_output/run_with_derived_metrics_v1/checkpoints/best_model_run_with_derived_metrics_v1.weights.h5'
T1C_PATH = '/content/drive/MyDrive/brats_goat_2024/MICCAI2024-BraTS-GoAT-TrainingData-With-GroundTruth/BraTS-GoAT-00000/BraTS-GoAT-00000-t1c.nii.gz'
T1N_PATH = '/content/drive/MyDrive/brats_goat_2024/MICCAI2024-BraTS-GoAT-TrainingData-With-GroundTruth/BraTS-GoAT-00000/BraTS-GoAT-00000-t1n.nii.gz'
T2F_PATH = '/content/drive/MyDrive/brats_goat_2024/MICCAI2024-BraTS-GoAT-TrainingData-With-GroundTruth/BraTS-GoAT-00000/BraTS-GoAT-00000-t2f.nii.gz'
T2W_PATH = '/content/drive/MyDrive/brats_goat_2024/MICCAI2024-BraTS-GoAT-TrainingData-With-GroundTruth/BraTS-GoAT-00000/BraTS-GoAT-00000-t2w.nii.gz'

# --- 2. BUILD MODEL AND LOAD WEIGHTS (This was the missing step) ---
print("🧠 Building model and loading weights...")
# You must use a custom_object_scope to load models with custom layers
with tf.keras.utils.custom_object_scope({'InstanceNormalization': InstanceNormalization}):
    # Build the model architecture (dropout is 0.0 for inference)
    model = nnunet_3d(dropout_rate=0.0)
    # Load your trained weights
    model.load_weights(MODEL_WEIGHTS_PATH)
print("✅ Model is ready.")

# --- 3. LOAD PATIENT DATA ---
print("📂 Loading patient data...")
t1c_nii = nib.load(T1C_PATH)
patient_affine = t1c_nii.affine
t1c_data = t1c_nii.get_fdata()
t1n_data = nib.load(T1N_PATH).get_fdata()
t2f_data = nib.load(T2F_PATH).get_fdata()
t2w_data = nib.load(T2W_PATH).get_fdata()

# --- 4. PREPROCESS & PREDICT ---
print("🚀 Running inference to get segmentation mask...")
input_tensor = preprocess_sample(t1c_data, t1n_data, t2f_data, t2w_data)
input_tensor = np.expand_dims(input_tensor, axis=0) # Add batch dimension

# The model outputs a list of 4 tensors; we need the first (main) one
pred_probs = model.predict(input_tensor)[0][0]
pred_labels = np.argmax(pred_probs, axis=-1)

# --- 5. CALCULATE VOLUMES ---
print("📏 Calculating tumor volumes...")
volumes = calculate_volumes(pred_labels, patient_affine)
print(f"Calculated Volumes: {volumes}")

# --- 6. GENERATE THE FINAL REPORT ---
print("\n📝 Calling Gemini to generate the report...")
final_report = generate_radiology_report(volumes)

print("\n" + "="*50)
print("          FINAL AI-GENERATED REPORT")
print("="*50)
print(final_report)

🧠 Building model and loading weights...


NameError: name 'InstanceNormalization' is not defined